In [ ]:
import pandas as pd
import seaborn as sns
from aequitas.group import Group
from aequitas.bias import Bias
from aequitas.fairness import Fairness
from aequitas.plotting import Plot

import warnings; warnings.simplefilter('ignore')

%matplotlib inline

In [ ]:
# Preprocess data to pass to Aequitas
#df = pd.DataFrame(columns=('entity_id', 'score', 'label_value'))
#pd.to

In [ ]:
#g = Group()
#xtab, _ = g.get_crosstabs(df)